<img src="images/logos/TClogo.png">

### in colaboration with

<img src="images/robotnik.png">

### presents

# ROS Developers Live Class n52

# How to localize a robot with GPS using the *robot_localization* package

This notebook is additional material for the **ROS Developers Live Class n.52** created and provided for free by **Alberto Ezquerro** and **Ricardo Tellez** of <a href="www.theconstructsim.com">The Construct</a>. You can distribute this notebook as long as you provide copy of this paragraph with it.

## Why this class?

In today's live class, we are going to learn the following contents:
1. Why to use a GPS for robot navigation
2. Why do we need other sensors on top of GPS
3. How to use the *robot_localization* package for GPS localization

Pre-requisites for this live class are:
* Basic knowledge of ROS concepts such as topics, publish and subscribe, ROS Service
* Know how to configure the *robot_localization* package for sensor fusion of **odometry** and **IMU**. <a href="https://www.youtube.com/watch?v=QZ5q59H2qaI">Check Live Class n51 if you don't know how</a>
* Love for Robotics 
* ...that's it!!

## Summit XL robot by Robotnik

<img src="images/summit_xl_outside.png">

For this class we are going to use the Summit XL robot by <a href="">Robotnik</a>. That is a robot that has the following characteristics:

* Four wheels
* Laser ranger
* IMU
* Camera
* GPS

In case you are interested on this robot, you can <a href="https://www.robotnik.eu/mobile-robots/summit-xl/">find more information here</a>.

<img src="images/Summitxl_real.jpg">

### How to use this ROSject

A <a href="http://rosjects.com">**ROSject**</a> is a **ROS project** packaged in such a way that all the material it contains (**ROS code, Gazebo simulations and Notebooks**) can be shared with any body **using only a web link**. That is what we did with all the attendants to the Live Class, we shared this ROSject with them (so they can have access to all the ROS material they contain).

**Check <a heref="https://www.youtube.com/watch?v=cR-Ow5K7oSo">this webinar</a> to learn more about ROSjects and how to create your own ROSjects**.

You will need to have a free account at the <a href="http://rosds.online">ROS Development Studio</a> (ROSDS). Get the account and then follow the indications below.

## 0. Let's launch Summit XL in the outside simulation with GPS plugin

For that you need to go to **Simulations->Select launch file**. Then select the only file available. After that you should get the following simulation

<img src="images/summit_xl_out2.png">

## 1. Why to use GPS for robot navigation?

Because you want to operate your robot in the outside, where there are minimal references that can allow you to create a map.

<img src="images/maxresdefault.jpg">

## 2. Why do we need odometry and IMU?

Because GPS data has some limitations:


* **GPS data is not very precise**: GPS data oscillates in value and your position will move a lot. This is unavoidable and you will have to mitigate that error in your own way. **Odometry** data will mitigate some of these errors, but not all of them because Summit XL wheels slide and this produces errors in the odometry. Bear that in mind.

* **GPS gives you no orientation**: GPS only gives you a point in space, no orientation. This means that you need to complement that with a **compass** of some kind. Here is where **IMU** comes in handy.

## 3. Using *robot_localization* package for GPS navigation

It is a package for mixing different sources of odometry into a more stable one.

<img src="images/robot_localization_graph3.png" width="600" />

### 3.1 Launch map server with an empty map

Because we want to use the **ROS Navigation stack** for GPS based navigation, we need to provide a map.

However, because we are in the outside, we are going to provide a fake map, that is, a completely empty map.

#### Create a package named *gps_navigation*

In [ ]:
$ roscd
$ cd ../src
$ catkin_create_pkg gps_navigation

#### Create the map data

In [ ]:
$ cd gps_navigation
$ mkdir maps
$ mkdir launch

Copy the empty map provided with this rosject, from the */home/user* to the *maps* directoy.

Then, create a *yaml* file named *mymap_empty.yaml* containing the configuration of that map.

In [ ]:
image:  mymap_empty.pgm
resolution: 0.050000
origin: [-50.000000, -50.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196

Finally, create a launch file named *start_map_server.launch* to launch the map server using that empty map:

In [ ]:
<launch>
    <arg name="map_file" default="$(find gps_navigation)/maps/mymap_empty.yaml"/>
    <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)" />
</launch>

### 3.2. Convert GPS data into Odometry data

We need to convert the GPS latitude and longitude data into XY coordinates that can be represented in 2D space.

For that we are going to use the **navsat_transform_node** ROS node provided by the **robot_localization** package.

#### Create the launch file *start_navsat.launch*

In [ ]:
<launch>
 <!-- -->
  <node pkg="robot_localization" type="navsat_transform_node" name="navsat_transform_node" respawn="true">

    <param name="magnetic_declination_radians" value="0"/>
    <param name="yaw_offset" value="0"/>
    <param name="zero_altitude" value="true"/>
    
    
    <param name="broadcast_utm_transform" value="false"/>
    <param name="publish_filtered_gps" value="false"/>

    
    <param name="use_odometry_yaw" value="false"/>
    <param name="wait_for_datum" value="false"/>
    

    <remap from="/imu/data" to="/imu/data" />
    <remap from="/gps/fix" to="/gps/fix" />
    <remap from="/odometry/filtered" to="/odom" />

  </node>

</launch>

* **magnetic_declination_radians**: This parameter is needed if your IMU provides its orientation with respect to the magnetic north. It states the magnetic declination for your location (the difference between the **magnetic north** and the **true north**). If you don’t know it, you can calculate it here http://www.ngdc.noaa.gov/geomag-web (make sure to convert the value to radians).


* **yaw_offset**: Your IMU should read 0 for yaw when facing east. If it doesn’t, enter the offset here (desired_value = offset + sensor_raw_value)


* **zero_altitude**: If this is set to true, the Odometry message produced by this node has its pose Z value set to 0.

* **broadcast_utm_transform**: This is in case you want to publish the static TF of the UTM (Origin of the GPS system) in the TF tree. 


* **publish_filtered_gps**: No mystery here, if you want it to publish improved odometry and IMU GPS data. If it's set to true, the *navsat_transform_node* will publish a sensor_msgs/NavSatFix message on the **/gps/filtered** topic.

* **use_odometry_yaw**: This is for when you have a very reliable odometry turning system. In the case of robots outdoors, the turning data in odometry is reliable enough because they skid only once in a while, especially on rough terrain. That's why it's turned OFF here.


* **wait_for_datum**: This is to give the system the datum directly, what is considered the origin of our GPS localization. Normally, it's the GPS that directly gives this data and that's why it's normaly turned off. If you need to give it for some reason, you will have to add the following param setting into this launch:

In [ ]:
<rosparam param="datum">[55.944904, -3.186693, 0.0, map, base_link]</rosparam>

### 3.3 Localize using GPS-odometry, robot odometry and IMU

We repeat now the process we did on the previous Live Class n.51

We use the **ekf_localization_node** from the **robot_localization** package to fuse the data of the **robot odometry**, **robot IMU** and **GPS odometry** generated with the **navsat_transform**.

#### Set the config file

Create a *config* directory in the package and create there the config file for *ekf_localization*

For that, we use a config file already provided to you in the */home/user* directory. File is called **gps_localization_config.yaml**

**IMPORTANT**: note, though, that we are setting the **publish_tf** variable to true. This is very important because now we need to publish the *transform* from **odom** to **map**.

#### Create a launch file for the *ekf_localization* node

Create a launch file named *ekf_localization.launch*:

In [ ]:
<launch>
    <!-- Run the ekf for map to odom config -->
    <node pkg="robot_localization" type="ekf_localization_node" name="ekf_localization_with_gps"> 
        <rosparam command="load" file="$(find gps_navigation)/config/gps_localization_config.yaml" />
    </node>
</launch>

### 3.4. Let's launch the whole navigation system based on GPS

We need to create a launch file that launches:

* The **map server** (to allow send goals)
* The **move_base** (to send the goals)
* The **GPS to odom** system
* The **localization system** based on the GPS odom data and the other sensors

Let's create that file with name *start_gps_nav.launch*:

In [ ]:
<launch>

    <include file="$(find gps_navigation)/launch/start_navsat.launch" />

    <!-- Run the ekf for map to odom config -->
    <include file="$(find gps_navigation)/launch/ekf_localization.launch" />

    <!-- Run the map server -->
    <include file="$(find gps_navigation)/launch/start_map_server.launch" />

    <!--- Run Move Base -->
    <include file="$(find my_move_base)/launch/my_move_base.launch" />

</launch>

In [ ]:
$ roslaunch gps_navigation start_gps_nav.launch

## 4. Launch Rviz to see the robot localized, and send goals to it

In [ ]:
$ rosrun rviz rviz

* Mission completed!!

### Before you log off, remember to <span style="background: #098be8; padding: 10px; color:white;">GIVE US A LIKE</span> and hit the <span style="background: #098be8; padding: 10px; color:white;">THUMBS UP</span> and <span style="background: #098be8; padding: 10px; color:white;">SUBSCRIBE</span> for more weekly tutorials!!!

We have an online academy that teaches you more about how to make robots navigate with ROS using GPS or laser sensors, as well as all the details of all the parameters of the configuration. Check the following related courses:
* <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/ros-robot-localization-package/">Fuse sensor data to improve localization</a>
* <a href="https://goo.gl/iog3x9">Mastering with ROS: Summit XL robot</a>


 <a href="https://goo.gl/7ApVAp"><img src="images/logos/RIAlogo.png"></a>

<img src="images/rosdevcon2.png">

A **hands on conference** where you will **learn and practice** at the same time **with the speakers**. 

It is an online conference with the same format of the Live Classes. You can attend from anywhere and will get a **rosject** with all the content of the speakers.

#### 8 speakers - 8 practical ROS projects on a single weekend

### You can also be a speaker of the conference. Check the call for papers

### More information here: <a href="http://www.rosdevcon.com">www.rosdevcon.com</a>

### Check the videos of the previous ROSDevCon 2018 <a href="">here</a>

# KEEP PUSHING YOUR ROS LEARNING!